In [1]:
import pandas as pd
import itertools
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import datetime as dt
import pickle

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import helpers as hp
from helpers import timer 
import model.queries as qrs

import os.path
from os import path

sns.set_theme(style="whitegrid")

In [3]:
pip install ipwhois

     |████████████████████████████████| 73 kB 4.3 MB/s  eta 0:00:01
     |████████████████████████████████| 208 kB 17.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install plotly

     |████████████████████████████████| 13.2 MB 16.6 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=c7b9cd54dbba54e6d98429694dfdfaa69fd7697621f927b5125dedf224ea904e
  Stored in directory: /root/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
def scan_gen(scan):
    while True:
        try:
            yield next(scan)['_source']
        except:
            break

def ps_trace_hashes(dt,src, dest,include=["timestamp","route-sha1","hops","destination_reached","src","dest","looping","path_complete"]):
    query = {
      "query" : {
        "bool" : {
          "must" : [
            {
              "range" : {
                "timestamp" : {
                  "gt" : dt[0],
                  "lte": dt[1]
                }
              }
            },
            {
              "term" : {
                'src' : {
                  "value" : src
                }
              }
            },
            {
              "term" : {
                'dest' : {
                  "value" : dest
                }
              }
            }
          ]
        }
      }
    }
#     print(str(query).replace("\'", "\""))
    try:
        return scan_gen(scan(hp.es,index="ps_trace",query=query, _source=include, filter_path=['_scroll_id', '_shards', 'hits.hits._source']))
    except Exception as e:
        print(e)
        
        
def getMeanLoss(dt, src, dest):
    q = {
          "size" : 0,
          "query" : {  
            "bool" : {
              "must" : [
                {
                  "range" : {
                    "timestamp" : {
                      "from" : dt[0],
                      "to" : dt[1]
                    }
                  }
                },
                {
                  "term" : {
                    "src" : src
                  }
                },
                {
                  "term" : {
                    "dest" : dest
                  }
                }
              ]
            }
          },
          "aggs": {
            "packet_loss": {
              "avg": {
                "field": "packet_loss"
              }
            }
          }
        }
#     print(str(q).replace("\'", "\""))
    
    data = hp.es.search(index='ps_packetloss', body=q, 
                _source=[ "packet_loss"])

    return data['aggregations']['packet_loss']['value']

    
def ps_packetloss(dt, src, dest, include=["timestamp","src","dest","packet_loss"]):
    query = {
      "query" : {
        "bool" : {
          "must" : [
            {
              "range" : {
                "timestamp" : {
                  "gt" : dt[0],
                  "lte": dt[1]
                }
              }
            },
            {
              "term" : {
                'src' : {
                  "value" : src
                }
              }
            },
            {
              "term" : {
                'dest' : {
                  "value" : dest
                }
              }
            }
          ]
        }
      }
    }
#     print(str(query).replace("\'", "\""))
    try:
        return scan_gen(scan(hp.es,index="ps_packetloss",query=query, _source=include, filter_path=['_scroll_id', '_shards', 'hits.hits._source']))
    except Exception as e:
        print(e)

        
def ps_trace(es, dt, src, dest):
    scan_gen = ps_trace_hashes(dt, src, dest)
    data = []
    hashes = {}
    records = 0

    for meta in scan_gen:
        data.append(meta)
#         print(meta)
        if 'route-sha1' in data[records]:
#         for doc in meta:
            hashes[meta['route-sha1']] = meta['hops']
        records += 1
        if not records % 500000:
            print(records)

    df = pd.DataFrame(data)
    if len(data)>0:
        df['dt'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

dt = hp.GetTimeRanges('2021-03-02 07:00','2021-03-12 07:00')

# dt = hp.GetTimeRanges('2021-03-01 07:01','2021-03-01 07:30')
print('Done')


Done


### Get all tested pairs

In [3]:
pairs = qrs.queryAllTestedPairs(dt)
pairsDf = pd.DataFrame(pairs)


### Get ps_trace data pair by pair and store all pairs in a list

In [4]:
tracedata = []
@timer
def getTraceData():
#     tracedata = []
    i = 0
    for src, dest in pairsDf[pairsDf['idx']=='ps_packetloss'][['src', 'dest']].values:
        data = ps_trace(hp.es, dt, src, dest)
        if len(data)>0:
#             tracedata.append({'src': src, 'dest': dest, 'data':data})
            tracedata.append(data)
        i+=1
    return tracedata

# save to a file
# tracedata = getTraceData()
# with open('/workspace/data/tracedata.pkl','wb') as outfile:
#     pickle.dump(tracedata,outfile)

# retrieve from file
with open("/workspace/data/tracedata.pkl", 'rb') as pickle_file:
    tracedata = pickle.load(pickle_file)

print('Number of pairs in ps_trace',len(tracedata))



Number of pairs in ps_trace 2832


### Calculate the packet loss between each two traceroute measures

In [5]:
# @timer
def processAndWrite2File(df, filename):
    df[['loss', 'path_changed']] = df.apply(calcLossBetweenMeasures, axis=1)
    with open(filename,"wb") as f:
        pickle.dump(df,f)
        
        
def calcLossBetweenMeasures(row):
    ploss = None
    pathChanged = 1
    if row['route-sha1'] == row['route-sha1_prev']:
        period = [row['timestamp_prev'], row['timestamp']]
        ploss = getMeanLoss(period, row['src'], row['dest'])
        pathChanged = 0

    return pd.Series([ploss, pathChanged])


def calculatePathLoss(trace_df):
    # sort data before calculating
    trace_df = trace_df.sort_values('timestamp', ascending=True)
    if 'route-sha1' in trace_df.columns:
        # get the previous row value
        trace_df['dt_prev'] = trace_df.dt.shift(1)
        trace_df['timestamp_prev'] = trace_df.timestamp.shift(1,fill_value=0)

        trace_df['route-sha1_prev'] = trace_df['route-sha1'].shift(1)

        filename = f"/workspace/data/dfs/{trace_df['src'].unique()[0]}-{trace_df['dest'].unique()[0]}.pkl"

        # write to a pickle file
        if path.exists(filename):
            if os.path.getsize(filename) > 0: 
                with open(filename, 'rb') as pickle_file:
                    ff = pickle.load(pickle_file)

                if len(ff) != len(trace_df):
#                     print('File not OK. Write!', len(ff), len(trace_df))
                    processAndWrite2File(trace_df, filename)
            else: processAndWrite2File(trace_df, filename)
        else:
            processAndWrite2File(trace_df, filename)


    return trace_df


#### Run the calculations in parallel

In [6]:
import multiprocessing

@timer
def addPacketLoss2Df():
    pool = multiprocessing.Pool()
    result = pool.map(calculatePathLoss, tracedata)
    return result

tracedata = addPacketLoss2Df()

Finished 'addPacketLoss2Df' in 5547.9736 secs


In [10]:
def saveData(df, file_name):
    with open(f"/workspace/data/{file_name}.pkl","wb") as f:
        pickle.dump(df,f)

#### Save the updated pair data

In [12]:
saveData(tracedata, 'tracedata_with_loss')

### Build a dictionarry of hashes, also merge all dataframes while looping

In [55]:
hashDict = {}
mergedDf = pd.DataFrame()

def getHashes(row):
    if 'route-sha1' in row:
        if row['route-sha1'] not in hashDict.keys():
#             print(row['hops'])
            hashDict[row['route-sha1']] = row['hops']

i = 0
for df in tracedata:
    df.apply(getHashes, axis=1)
    mergedDf = mergedDf.append(df)
    i+=1

print('Number of unique hashes:', len(hashDict))
print('Total number of measures:', len(mergedDf))

saveData(hashDict, 'hashDict')
saveData(mergedDf, 'mergedDf')

Number of unique hashes: 2792
Total number of measures: 5098


In [57]:
# hashDict

### Build a dictionary of all hops

In [17]:
hopsDict = {}

for hashKey, hops in list(hashDict.items()):
    for hop in hops:
        temp = []
        if hop in hopsDict.keys():
            temp = hopsDict[hop]['hashes']
#             if len(temp) > 300:
            if hashKey not in temp:
                temp.append(hashKey)
#             print(len(temp))
            hopsDict[hop] = {'hashes': temp, 'num_of_unique_hashes': len(temp)}
        else: hopsDict[hop] = {'hashes': [hashKey], 'num_of_unique_hashes': 1}
#         print(hop, temp)

print(f'Number of unique hops: {len(hopsDict)}')
saveData(hopsDict, 'hopsDict')

Number of unique hops: 2302


### Create a hash dataframe and encode the hops

In [62]:
cols = ['hash']
cols = cols + list(hopsDict.keys())

hashDf = pd.DataFrame(columns=cols)
hashDf['hash'] = hashDict.keys()
hashDf[cols[1:]] = 0

for hop in hopsDict.keys():
    hashDf[hop] =  np.where(hashDf['hash'].isin(hopsDict[hop]['hashes']), 1, 0)

In [64]:
hashDf

,hash,109.105.124.65,109.105.97.57,109.105.97.51,192.16.166.86,192.16.166.85,192.16.166.213,192.16.166.2,117.103.111.233,117.103.105.191,...,72.36.80.78,72.36.80.30,146.57.252.194,146.57.252.222,146.57.252.225,146.57.253.217,129.93.6.177,129.93.5.165,72.36.80.80,72.36.96.254
0,e64b202e37888601fca7863026cb9f2bb8fc5f8a,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2a64d0017fe4bdf740ef778913b58508c575e000,1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,6f922ee6b569a14699906d7f687a6149235bab47,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ef8add2f0a2bb52c41c2899777939ac584fcd477,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,e8a72e214be0dbfa37c68abf7bd136a00b84ec77,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,e4823fa9386f56d71c2e3e3b2bbdf36134358e8a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2788,1100059b5eb4d7ce2b1dc9b3fd629654095c026c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2789,30ad77626c69497c2e0c335578eff5485d35882e,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,0
2790,fda2fffc92f9383712946b02f3065af9c541164a,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0


In [66]:
saveData(hashDf, 'hashDf')

In [67]:
hopsDict = {}

for hashKey, hops in list(hashDict.items()):
    for hop in hops:
        temp = []
        if hop in hopsDict.keys():
            temp = hopsDict[hop]['hashes']
#             if len(temp) > 300:
            if hashKey not in temp:
                temp.append(hashKey)
#             print(len(temp))
            hopsDict[hop] = {'hashes': temp, 'num_of_unique_hashes': len(temp)}
        else: hopsDict[hop] = {'hashes': [hashKey], 'num_of_unique_hashes': 1}
#         print(hop, temp)

print(f'Number of unique hops: {len(hopsDict)}')
# hopsDict

Number of unique hops: 2302


### Create hops dataframe
Add the mean loss for each hop as well as the sum of all packet loss, both based on the paths in which a specific hop takes part

In [68]:
hopsDf = pd.DataFrame(hopsDict).T.reset_index()

aggPathDf = mergedDf.groupby('route-sha1').agg(loss_mean=('loss', 'mean'), loss_sum=('loss', 'sum')).reset_index()
def addMeanLoss2Hops(row):
    return aggPathDf[aggPathDf['route-sha1'].isin(row['hashes'])]['loss_mean'].mean()

def addAccuLoss2Hops(row):
    return aggPathDf[aggPathDf['route-sha1'].isin(row['hashes'])]['loss_sum'].sum()

hopsDf['mean_loss']=hopsDf.apply(lambda row:addMeanLoss2Hops(row),axis=1)
hopsDf['accu_loss']=hopsDf.apply(lambda row:addAccuLoss2Hops(row),axis=1)

saveData(hopsDf, 'hopsDf')

In [69]:
hopsDf.head()

,index,hashes,num_of_unique_hashes,mean_loss,accu_loss
0,109.105.124.65,"[e64b202e37888601fca7863026cb9f2bb8fc5f8a, 2a6...",17,0.000007,0.000139
1,109.105.97.57,"[e64b202e37888601fca7863026cb9f2bb8fc5f8a, 2a6...",7,0.000017,0.000139
2,109.105.97.51,"[e64b202e37888601fca7863026cb9f2bb8fc5f8a, 2a6...",9,0.000014,0.000139
3,192.16.166.86,"[e64b202e37888601fca7863026cb9f2bb8fc5f8a, 2a6...",3,0.000000,0.000000
4,192.16.166.85,"[e64b202e37888601fca7863026cb9f2bb8fc5f8a, 2a6...",3,0.000000,0.000000


### Check the distribution of number of unique hashes

In [33]:
fig = px.histogram(hopsDf, nbins=50, x="num_of_unique_hashes")
py.offline.plot(fig)

'temp-plot.html'

### Number of hashes to mean packet loss relationship

In [70]:
hopsDf = hopsDf.sort_values('num_of_unique_hashes', ascending=False)

fig = px.scatter(hopsDf, x='mean_loss', y='num_of_unique_hashes')
fig.update_layout(title=f'Num of hashes to mean packet loss relationship' , template='plotly_white')
py.offline.plot(fig)

'temp-plot.html'

In [38]:
hopsDf.sort_values('accu_loss')

,index,hashes,num_of_unique_hashes,mean_loss,accu_loss
30,192.16.166.158,"[5c04fb2256f671a8ef44d7eb9fc77f5597a0241a, 800...",7,0.000000,0.000000
2153,2001:468:fc:74::2,"[42d3ac59b76c34a6253a3ae6a471612e54f8175b, c3a...",3,NaN,0.000000
1258,2001:1458:a00:5::2,"[85204832a4778f12ca64b77e5c8b8309a6738c16, 288...",3,NaN,0.000000
1269,2001:18e8:804:ff05::1,"[1ca6a6a51b4a125c65e61d9e00513159d435b6a7, c4d...",3,NaN,0.000000
1262,2001:1458:301:86::2,"[2882b30717740b88734f3d794c2066af62e32646, 4c8...",3,NaN,0.000000
...,...,...,...,...,...
164,146.97.33.22,"[532cceda7cdd5f91dd6cdb6c22ae23f73d87fc60, 876...",104,0.132080,12.000468
280,146.97.33.42,"[876b0add8ece31feb403851cc396d2f15ff94893, 297...",103,0.160005,14.000468
210,146.97.36.222,"[7fd254eded45bd8c2ee40e906a8285c716d212c0, 40f...",58,0.476197,16.000337
154,143.167.3.116,"[9804a1959daffa9e0a115d2d429709ef2b3aefb6, 95f...",27,1.000000,19.000000
